In [1]:
pip install openrouteservice


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import openrouteservice as ors

In [3]:
API_key = '5b3ce3597851110001cf62487fdbd58fccf548529a19d99f12b4692d'

In [4]:
repo_url='https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
column_names = ['commune', 'NPA', '',"Commune d'annonce","Commune d'annonce référence",'canton','E','N', 'langue','date_validite']  # example
vaud_cities = pd.read_excel(repo_url+'vaud_communes.xlsx',header=None, names=column_names)
display(vaud_cities)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,commune,NPA,,Commune d'annonce,Commune d'annonce référence,canton,E,N,langue,date_validite
Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,1 an,2 ans,3 ans,4 ans,5 ans,6 ans,7 ans,8 ans,9 ans,10 ans,11 ans,12 ans,13 ans,14 ans,15 ans,16 ans,17 ans,18 ans,19 ans,20 ans,21 ans,22 ans,23 ans,24 ans,25 ans,26 ans,27 ans,28 ans,29 ans,30 ans,31 ans,32 ans,33 ans,34 ans,35 ans,36 ans,37 ans,38 ans,39 ans,40 ans,41 ans,42 ans,43 ans,44 ans,45 ans,46 ans,47 ans,48 ans,49 ans,50 ans,51 ans,52 ans,53 ans,54 ans,55 ans,56 ans,57 ans,58 ans,59 ans,60 ans,61 ans,62 ans,63 ans,64 ans,65 ans,66 ans,67 ans,68 ans,69 ans,70 ans,71 ans,72 ans,73 ans,74 ans,75 ans,76 ans,77 ans,78 ans,79 ans,80 ans,81 ans,82 ans,83 ans,84 ans,85 ans,86 ans,87 ans,88 ans,89 ans,90 ans,91 ans,92 ans,93 ans,94 ans,95 ans,96 ans,97 ans,98 ans,99 ans,100+ ans
Aigle,1860,Aigle,5401,VD,2563821.020,1129273.277,fr,11780,138,131,124,141,141,147,139,158,140,125,141,123,116,141,144,137,137,135,123,119,133,141,132,143,162,163,169,176,163,159,178,179,203,197,171,193,170,168,171,148,151,146,161,155,147,150,142,160,153,163,135,157,168,152,144,164,126,161,150,149,146,147,144,122,124,118,101,125,94,101,108,108,90,81,80,89,82,95,94,89,73,64,74,57,43,43,32,19,30,34,25,20,25,16,6,9,7,0,3,5,4
Le Sépey,1863,Aigle,5401,VD,2568544.893,1131955.818,fr,11780,138,131,124,141,141,147,139,158,140,125,141,123,116,141,144,137,137,135,123,119,133,141,132,143,162,163,169,176,163,159,178,179,203,197,171,193,170,168,171,148,151,146,161,155,147,150,142,160,153,163,135,157,168,152,144,164,126,161,150,149,146,147,144,122,124,118,101,125,94,101,108,108,90,81,80,89,82,95,94,89,73,64,74,57,43,43,32,19,30,34,25,20,25,16,6,9,7,0,3,5,4
Bex,1880,Bex,5402,VD,2567230.358,1121779.532,fr,8806,79,68,88,104,88,112,115,108,110,120,110,110,126,116,95,119,99,105,90,101,82,96,103,93,107,112,104,109,102,111,111,140,127,114,144,115,154,129,119,121,123,137,115,107,119,115,121,115,98,129,107,87,130,110,102,108,127,107,136,120,109,116,106,116,102,97,97,95,82,66,82,70,60,50,74,66,64,68,65,56,60,54,40,44,39,47,31,31,28,27,23,14,11,13,14,8,6,2,0,1,3
Frenières-sur-Bex,1880,Bex,5402,VD,2571040.657,1121183.948,fr,8806,79,68,88,104,88,112,115,108,110,120,110,110,126,116,95,119,99,105,90,101,82,96,103,93,107,112,104,109,102,111,111,140,127,114,144,115,154,129,119,121,123,137,115,107,119,115,121,115,98,129,107,87,130,110,102,108,127,107,136,120,109,116,106,116,102,97,97,95,82,66,82,70,60,50,74,66,64,68,65,56,60,54,40,44,39,47,31,31,28,27,23,14,11,13,14,8,6,2,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villars-Epeney,1404,Villars-Epeney,5935,VD,2543412.650,1181535.124,fr,108,1,2,2,0,0,3,2,1,2,1,1,3,1,2,4,1,1,1,2,1,2,0,0,0,1,0,0,1,2,1,2,1,1,1,6,2,1,2,1,0,4,2,0,0,2,1,0,0,3,1,0,1,1,2,1,2,1,1,0,3,3,6,0,1,1,3,1,3,1,0,1,0,4,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Vugelles-La Mothe,1431,Vugelles-La Mothe,5937,VD,2534182.487,1186399.984,fr,147,3,1,1,1,3,1,2,1,0,0,2,4,2,2,2,3,4,1,3,1,2,1,1,1,2,0,1,1,1,2,1,0,4,3,2,0,2,5,0,2,1,5,3,0,2,2,1,2,3,2,0,0,4,2,4,5,1,6,4,4,1,1,2,0,3,1,2,2,3,1,1,1,0,2,1,0,0,1,0,0,0,1,0,1,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0
Yverdon-les-Bains,1400,Yverdon-les-Bains,5938,VD,2539180.581,1180801.624,fr,30332,245,307,327,316,288,334,303,348,350,312,368,304,368,358,343,355,340,339,339,367,354,324,341,358,377,396,419,393,431,439,490,472,457,450,431,403,441,432,432,468,433,398,422,411,418,484,442,408,387,406,450,387,411,415,409,386,388,380,356,379,347,352,342,334,3

In [5]:
coordonnees_venoge = [46.60396069250175, 6.538034286509177]

In [6]:
import math
# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

        # Convert CH y/x to WGS long
def CHtoWGSlng( y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
        + (0.791484 * y_aux * x_aux) + \
        + (0.1306 * y_aux * pow(x_aux, 2))) + \
        - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng

In [7]:
vaud_cities['lat'] = vaud_cities.apply(lambda x: CHtoWGSlat(x['E_lv03'], x['N_lv03']), axis=1)
vaud_cities['lng'] = vaud_cities.apply(lambda x: CHtoWGSlng(x['E_lv03'], x['N_lv03']), axis=1)


KeyError: 'E_lv03'

In [ ]:
vaud_cities['E_lv03'] = vaud_cities['E']-2000000
vaud_cities['N_lv03'] = vaud_cities['N']-1000000

In [ ]:
vaud_cities

,commune,NPA,,Commune d'annonce,Commune d'annonce référence,canton,E,N,langue,date_validite,lat,lng,E_lv03,N_lv03
0,Aigle,1860,0,Aigle,5401,VD,2563821.020,1129273.277,fr,2008-07-01,46.313885,6.968954,563821.020,129273.277
1,Le Sépey,1863,0,Aigle,5401,VD,2568544.893,1131955.818,fr,2008-07-01,46.338253,7.030098,568544.893,131955.818
2,Bex,1880,0,Bex,5402,VD,2567230.358,1121779.532,fr,2008-07-01,46.246650,7.013740,567230.358,121779.532
3,Frenières-sur-Bex,1880,2,Bex,5402,VD,2571040.657,1121183.948,fr,2008-07-01,46.241467,7.063181,571040.657,121183.948
4,Fenalet-sur-Bex,1880,3,Bex,5402,VD,2568662.358,1125041.373,fr,2008-07-01,46.276060,7.032088,568662.358,125041.373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,Villars-Epeney,1404,2,Villars-Epeney,5935,VD,2543412.650,1181535.124,fr,2008-07-01,46.782581,6.697615,543412.650,181535.124
541,Vugelles-La Mothe,1431,0,Vugelles-La Mothe,5937,VD,2534182.487,1186399.984,fr,2008-07-01,46.825492,6.576052,534182.487,186399.984
542,Yverdon-les-Bains,1400,0,Yverdon-les-Bains,5938,VD,2539180.581,1180801.624,fr,2008-07-01,46.775610,6.642296,539180.581,180801.624
543,Gressy,1432,2,Yverdon-les-Bains,5938,VD,2538768.002,1178103.684,fr,2008-07-01,46.751304,6.637255,538768.002,178103.684


In [ ]:
vaud_cities['distance_to_venoge_km'] = None

In [ ]:
city=f"{vaud_cities[vaud_cities['commune'] == 'Aigle']['lng'].iloc[0]},{vaud_cities[vaud_cities['commune'] == 'Aigle']['lat'].iloc[0]}"
print(city)

6.968953748717655,46.31388497394788


In [ ]:
client = ors.Client(key=API_key)
route = client.directions(
    profile='driving-car',
    format='geojson',
    coordinates=[[6.968953, 46.313884],[6.5367833784414175, 46.60314203414339]]
)
print(route)
print(route['features'][0]['properties']['segments'][0]['distance'])

ApiError: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 46.3138840 6.9689530.; Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 46.6031420 6.5367834.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1746460310616}})

In [ ]:
#call=requests.get(f'https://api.openrouteservice.org/v2/directions/driving-car?api_key={API_key}&start={city}&end={coordonnees_venoge},headers=headers')
#if call.status_code == 200:
#    data = call.json()
#    distance = data['routes'][0]['summary']['distance'] / 1000
#    print(distance)

In [ ]:
from tqdm import tqdm
headers = {
    'Accept': 'application/json',
    'Authorization': API_key  # This is required for ORS v2
}

# Loop through each city in the DataFrame
for index, row in tqdm(vaud_cities.iterrows(), total=vaud_cities.shape[0]):
    city_lng_lat = f"{row['lng']},{row['lat']}"  # must be lng,lat

    url = f"https://api.openrouteservice.org/v2/directions/driving-car"
    params = {
        'start': city_lng_lat,
        'end': '46.60396069250175,6.538034286509177',  # Venoge coordinates
    }

    try:
        response = requests.get(f'https://api.openrouteservice.org/v2/directions/driving-car?api_key={API_key}&start={city_lng_lat}&end=46.60396069250175,6.538034286509177')
        if response.status_code == 200:
            data = response.json()
            distance_m = data['features'][0]['properties']['segments'][0]['distance']
            vaud_cities.at[index, 'distance_to_venoge_km'] = round(distance_m / 1000, 2)
        else:
            print(f"Failed for {row['commune']} — {response.status_code}")
            vaud_cities.at[index, 'distance_to_venoge_km'] = None
    except Exception as e:
        print(f"Error at row {index}: {e}")
        df.at[index, 'distance_to_venoge_km'] = None

  0%|          | 0/545 [00:00<?, ?it/s]

  0%|          | 1/545 [00:00<02:03,  4.41it/s]

Failed for Aigle — 404


  1%|          | 3/545 [00:00<02:08,  4.23it/s]

Failed for Le Sépey — 404
Failed for Bex — 404


...

 67%|██████▋   | 366/545 [00:36<00:14, 12.57it/s]

Failed for Chavornay — 429
Failed for Corcelles-sur-Chavornay — 429
Failed for Essert-Pittet — 429


 68%|██████▊   | 368/545 [00:37<00:17,  9.92it/s]


Failed for Les Clées — 429


KeyboardInterrupt: 